# RAG 的使用

In [3]:
from dotenv import load_dotenv

load_dotenv()

True

In [8]:
from operator import itemgetter

from langchain_community.vectorstores.faiss import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import OpenAIEmbeddings, ChatOpenAI

In [10]:
vector_store = FAISS.from_texts(
    ["harrison worked at kensho"], embedding=OpenAIEmbeddings()
)
retriever = vector_store.as_retriever()
template =  """Answer the question base on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template=template)
model = ChatOpenAI()

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

chain.invoke("where did harrison work?")

'Harrison worked at Kensho.'

处理入参是字典的情况

In [15]:
template1 =  """Answer the question base on the following context:
{context}

Question: {question}

Answer in the following language: {language}
"""
prompt1 = ChatPromptTemplate.from_template(template=template1)
model1 = ChatOpenAI()

chain1 = (
    {
        "context": itemgetter("question") | retriever, 
        "question": itemgetter("question"),
        "language": itemgetter("language")
    }
    | prompt1
    | model1
    | StrOutputParser()
)

chain1.invoke({"question": "where did harrison work", "language": "chinese"})

'哈里森在肯肖工作。'

## 对话文档检索

In [17]:
from langchain.schema import format_document
from langchain_core.messages import AIMessage, HumanMessage, get_buffer_string
from langchain_core.runnables import RunnableParallel
from langchain_core.prompts import PromptTemplate

template2 = """Answer the following conversation and a follow up question, rephrase the follow up question to be aa standlone question, in its original language.

Chat History:
{chat_history}
Follow Up Question: {question}
Standlone question:
"""
condense_question_prompt = PromptTemplate.from_template(template2)

template_answer = """Answer the question based only on the following context:
{context}

Question: {question}
"""

answer_prompt = ChatPromptTemplate.from_template(template_answer)

# 需要搞明白为什么这么写
default_doc_prompt = PromptTemplate.from_template("{page_content}")

def _combine_docs(docs, doc_prompt=default_doc_prompt, doc_separator="\n\n"):
    docs_strings = [format_document(doc, doc_prompt) for doc in docs]
    print(docs_strings)
    return doc_separator.join(docs_strings)

_inputs = RunnableParallel(
    standalone_question=RunnablePassthrough.assign(
        chat_history=lambda x: get_buffer_string(x["chat_history"])
    )
    | condense_question_prompt
    | ChatOpenAI()
    | StrOutputParser()
)

_context = {
    "context": itemgetter("standalone_question") | retriever | _combine_docs,
    "question": lambda x: x["standalone_question"]
}

# input 根据对话历史总结问题
# context 搜索到的文档信息和问题
# 结合回答
chat_rag_chain = _inputs | _context | answer_prompt | ChatOpenAI()

chat_rag_chain.invoke(
    {
        "question": "where did harrison work?",
        "chat_history": []
    }
)

['harrison worked at kensho']


AIMessage(content='Harrison worked at Kensho.')